In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%display default

$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
# How to implement new algebraic structures in Sage: Categories and coercion

> *Simon King* *Friedrich-Schiller-Universität Jena* *E-mail: simon dot king at uni hyphen jena dot de* *© 2019*

The aim of this tutorial is to explain how one can benefit from Sage's category framework and coercion model when implementing new algebraic structures.

As an illustration, we implement fraction fields.

## Outline

#### Use existing base classes

<p style="margin-left: 30px"> There are sub-classes of sage.structure.parent.Parent resp. of sage.structure.element.Element that will help you a lot. Inheriting from these classes is essential for using Sage's coercion system.</p>

<p style="margin-left: 30px"> Arithmetic operations should be implemented by *single underscore* methods, such as _add_, _mul_.</p>

#### Turn your parent structure into an object of a category

<p style="margin-left: 30px"> Declare the category during initialisation - Your parent structure will inherit further useful methods and consistency tests.</p>

#### Provide your parent structure with an element class

<p style="margin-left: 30px"> Assign to it an attribute called "Element" - The elements will inherit further useful methods from the category.</p>

<p style="margin-left: 30px"> In addition, some basic conversions will immediately work.</p>

#### Implement further conversions

<p style="margin-left: 30px"> Never override a parent's __call__ method! Provide _element_constructor_ instead.</p>

#### Declare coercions

<p style="margin-left: 30px"> If a conversion happens to be a morphism, you may consider to turn it into a coercion. It will then *implicitly* be used in arithmetic operations.</p>

#### Advanced coercion: Define construction functors for your parent structure

<p style="margin-left: 30px">Sage will automatically create new parents for you when needed, by some kind of "pushout" construction.</p>

#### Run the automatic test suites

<p style="margin-left: 30px">Each method should be documented and provide a doc test (we are not giving examples here). In addition, any method defined for a category should be supported by a test method that is executed when running the test suite.</p>

## Base classes

In Sage, a "Parent" is an object of a category and contains elements.

Parents should inherit from $sage.structure.parent.Parent$ and their elements from $sage.structure.element.Element$.

Sage provides appropriate sub-classes of Parent and Element for a variety of more concrete algebraic structures, such as groups, rings, or fields, and of their elements.

### The parent

We want to implement a field, and we should thus start with the base class `sage.rings.field.Field`.

In [2]:
from sage.rings.ring import Field















Let us compare the methods provided by that class with the methods provided by a general parent:

In [3]:
[p for p in dir(Field) if p not in dir(Parent)]

['__fraction_field',
 '__ideal_monoid',
 '__iter__',
 '__len__',
 '__pow__',
 '__rpow__',
 '__rtruediv__',
 '__rxor__',
 '__truediv__',
 '__xor__',
 '_an_element_impl',
 '_coerce_',
 '_coerce_c',
 '_coerce_impl',
 '_coerce_try',
 '_default_category',
 '_gens',
 '_ideal_class_',
 '_latex_names',
 '_list',
 '_one_element',
 '_pseudo_fraction_field',
 '_random_nonzero_element',
 '_unit_ideal',
 '_zero_element',
 '_zero_ideal',
 'algebraic_closure',
 'base_extend',
 'class_group',
 'coerce_map_from_c',
 'content',
 'derivation',
 'derivation_module',
 'divides',
 'epsilon',
 'extension',
 'fraction_field',
 'frobenius_endomorphism',
 'gcd',
 'gen',
 'gens',
 'has_coerce_map_from_c',
 'ideal',
 'ideal_monoid',
 'integral_closure',
 'is_commutative',
 'is_field',
 'is_integral_domain',
 'is_integrally_closed',
 'is_noetherian',
 'is_prime_field',
 'is_ring',
 'is_subring',
 'krull_dimension',
 'ngens',
 'one',
 'order',
 'prime_subfield',
 'principal_ideal',
 'quo',
 'quotient',
 'quotient_r

Any ring in Sage has a **base** and a **base ring**.

The "usual" fraction field of a ring R has the base R and the base ring R.base\_ring() - we'll do the same:

In [4]:
Frac(QQ['x']).base(), Frac(QQ['x']).base_ring()

(Univariate Polynomial Ring in x over Rational Field, Rational Field)

#### Declaring the base of a field
Easy. Just pass it as an argument.

In [5]:
Field(ZZ['x']).base()

Univariate Polynomial Ring in x over Integer Ring

Python uses double-underscore methods for arithemetic methods and string representations. Sage's base classes often have a default implementation.

Hence, **implement SINGLE underscore methods \_repr\_, and similarly \_add\_, \_mul\_ etc.**

You may consider to **make your parent "unique"** (parents should only evaluate equal if they are identical)

Here, we also want to test whether the base is an integral domain (this is another use case of categories).

Last, we add a base\_ring method (as mentioned above) and a method that returns the characteristic of the field.

In [6]:
class MyFrac(UniqueRepresentation, Field):
    def __init__(self, base):
        if base not in IntegralDomains():
            raise ValueError("%s is no integral domain" % base)
        Field.__init__(self, base)
    def _repr_(self):
        return "NewFrac(%s)"%repr(self.base())
    def base_ring(self):
        return self.base().base_ring()
    def characteristic(self):
        return self.base().characteristic()

In [7]:
MyFrac(ZZ['x'])

NewFrac(Univariate Polynomial Ring in x over Integer Ring)

In [8]:
MyFrac(Integers(15))

ValueError: Ring of integers modulo 15 is no integral domain

Note that UniqueRepresentation automatically provides pickling, at least to some extent.

In [9]:
loads(dumps(MyFrac(ZZ))) is MyFrac(ZZ)

True

### The elements

We use the base class sage.structure.element.FieldElement. Note that field elements do not require that their parent is a field:

In [10]:
from sage.structure.element import FieldElement
FieldElement(ZZ)

Generic element of a structure

**Implementation idea**

* A fraction field element should have numerator and denominator in the base ring.<br>
  *Caveat:* We have numerator/denominator *methods* in Sage, while Python says it should be *properties*

* The denominator must not be zero - by default, the denominator is one.

* String representation via \_repr\_. **Do not override the default double underscore \_\_repr\_\_!**

* Arithmetic via \_add\_, \_mul\_. **Do not override the default double underscore \_\_add\_\_, \_\_mul\_\_!** <br>Otherwise, you won't have coercion.

In the single underscore methods and in \_\_cmp\_\_, you can always assume that **both arguments belong to the same parent**.

Note the use of self.\_\_class\_\_ in the arithmetic methods. This is important, as later we will in fact work with *sub-classes* of MyElement. In order to make this notebook Python 3 compliant, we implement rich comparison.

In [11]:
from sage.structure.richcmp import richcmp

In [12]:
class MyElement(FieldElement):
    def __init__(self, parent, n, d = None):
        B = parent.base()
        if d is None:
            d = B.one()
        if n not in B or d not in B:
            raise ValueError("Numerator and denominator must be elements of %s"%B)
        # Numerator and denominator should not just be "in" B,
        # but should be defined as elements of B
        d = B(d)
        n = B(n)
        if d==0:
            raise ZeroDivisionError("The denominator must not be zero")
        if d<0:
            self.n = -n
            self.d = -d
        else:
            self.n = n
            self.d = d
        FieldElement.__init__(self,parent)
    def numerator(self):
        return self.n
    def denominator(self):
        return self.d
    def _repr_(self):
        return "(%s):(%s)"%(self.n,self.d)
    def _richcmp_(self, other, op):
        return richcmp(self.n*other.denominator(), other.numerator()*self.d, op)
    def _add_(self, other):
        C = self.__class__
        D = self.d*other.denominator()
        return C(self.parent(), self.n*other.denominator()+self.d*other.numerator(), D)
    def _sub_(self, other):
        C = self.__class__
        D = self.d*other.denominator()
        return C(self.parent(), self.n*other.denominator()-self.d*other.numerator(),D)
    def _mul_(self, other):
        C = self.__class__
        return C(self.parent(), self.n*other.numerator(), self.d*other.denominator())
    def _div_(self, other):
        C = self.__class__
        return C(self.parent(), self.n*other.denominator(), self.d*other.numerator())

Thanks to the single underscore methods, some basic arithmetics works, **if** we stay inside our parent structure:

In [13]:
P = MyFrac(ZZ)
a = MyElement(P, 3,4)
b = MyElement(P, 1,2)

In [14]:
a+b, a-b, a*b, a/b

((10):(8), (2):(8), (3):(8), (6):(4))

In [15]:
a-b == MyElement(P, 1,4)
a-b < a*b

True

True

We didn't implement exponentiation - but it just works:

In [16]:
a**3

(27):(64)

There is a default implementation of element tests. We can already do

In [17]:
a in P

True

since a is defined as an element of P.

But we can not verify yet that the integers are contained in the fraction field of the ring of integers:

In [18]:
1 in P

NotImplementedError: cannot construct elements of NewFrac(Integer Ring)

Unfortunately the error message isn't exactly helpful. Anyway, we will implement that later.

## Chosing a category

Sometimes the base classes do not reflect the mathematics:

The set of $m\times n$ matrices over a field forms, in general, not more than a vector space. Hence, they are not implemented on top of sage.rings.ring.Ring.

However, if $m=n$ then the matrix space is an algebra, thus, in particular is a ring. From the point of view of Python classes, both are the same:

In [19]:
MS1 = MatrixSpace(QQ,2,3)
isinstance(MS1, Ring)

False

In [20]:
MS2 = MatrixSpace(QQ,2)
isinstance(MS2, Ring)

False

Sage's category framework can differentiate the two cases:

In [21]:
Rings()

Category of rings

In [22]:
MS1 in Rings()

False

In [23]:
MS2 in Rings()

True

Surprisingly, MS2 has *more* methods than MS1:

In [24]:
import inspect
len([s for s in dir(MS1) if inspect.ismethod(getattr(MS1,s,None))])

81

In [25]:
len([s for s in dir(MS2) if inspect.ismethod(getattr(MS2,s,None))])

119

In the past, it used to be the case that the classes of MS1 and MS2 are the same. This has changed:

In [26]:
MS1.__class__ is MS2.__class__

False

It is no surprise that our parent P defined above knows that it belongs to the category of fields, as it is derived from the base class of fields.

In [27]:
P.category()

Category of fields

However, we will choose a smaller category, namely the category of quotient fields.

### Why should one choose a category?

One can provide default **methods** ***for all objects*** of a category , and ***for all elements*** of such objects. Hence:

* It is an alternative way of inheriting useful stuff.

* It does not rely on implementation details, but on mathematical concepts.

* In addition, the categories define **test suites** for their objects and elements. Hence, one also gets basic sanity tests for free.

How does this so-called *category framework* work?

* Abstract base classes for the objects ("parent\_class") and the elements of objects ("element\_class") are provided by attributes of the category.

* At initialisation of a parent, the class of the parent is *dynamically changed* into a sub-class of the category's parent class.

* The parent is also provided with a sub-class of the category's element class (see below).

Dynamic change of classes does not work in Cython. Nevertheless, method inheritance still works, by virtue of a \_\_getattr\_\_ method.

**$\Rightarrow$ It is strongly recommended to use the category framework both in Python and in Cython.**

Let us see whether there is any gain in chosing the category of quotient fields instead of the category of fields:

In [28]:
QuotientFields().parent_class
QuotientFields().element_class

<class 'sage.categories.quotient_fields.QuotientFields.parent_class'>

<class 'sage.categories.quotient_fields.QuotientFields.element_class'>

In [29]:
[p for p in dir(QuotientFields().parent_class) if p not in dir(Fields().parent_class)]

[]

In [30]:
[p for p in dir(QuotientFields().element_class) if p not in dir(Fields().element_class)]

['_derivative',
 'denominator',
 'derivative',
 'numerator',
 'partial_fraction_decomposition']

So, there is no immediate gain for the parent, but there useful default implementations of methods for the elements.

### Category framework for the parent

Declare the correct category by an optional argument of the field constructor:

In [ ]:
from sage.categories.quotient_fields import QuotientFields
Category.join([QuotientFields(), FiniteSets()])

In [31]:
class MyFrac(MyFrac):
    def __init__(self, base, category=None):
        if base not in IntegralDomains():
            raise ValueError("%s is no integral domain"%base)
        Field.__init__(self, base, 
                       category=Category.join([category, QuotientFields()]) if category else QuotientFields())

When constructing instances of MyFrac, their class is dynamically changed into a new class MyFrac\_with\_category.

It is a common sub-class of MyFrac and of the category's parent class.

In [32]:
P = MyFrac(ZZ)
type(P)
P.category()
isinstance(P,MyFrac)
isinstance(P,QuotientFields().parent_class)

<class '__main__.MyFrac_with_category'>

Category of quotient fields

True

True

P inherits additional methods:

The base class `Field` does not have a method `sum`. But P inherits such method from the category of commutative additive monoids

In [33]:
P.sum.__module__

'sage.categories.additive_monoids'

However, it does not work, yet, because it relies on some other stuff that we will implement later:

In [34]:
a = MyElement(P, 3,4)
b = MyElement(P, 1,2)
c = MyElement(P, -1,2)
P.sum([a,b,c])

NotImplementedError: cannot construct elements of NewFrac(Integer Ring)

### Category framework for the elements

Again, the class is dynamically changed into a sub-class of the element class of a category (Cython mimmicks that by a `__getattr__` method)

However, the category framework is invoked in a different way for elements than for parents.

* Provide the parent P (or its class) with an attribute "**P.Element**", whose value is a class.

* The parent\**automatically*\* obtains an attribute "**P.element\_class**", that subclasses both P.Element with P.category().element\_class.

**For providing our fraction fields with their own element classes, we just need to add a single line to our class:**

In [35]:
class MyFrac(MyFrac):
    Element = MyElement

That little change means:

* We can now create elements by simply calling the parent
* We can suddenly use a method zero\_element
* The "sum" method works

In [36]:
P = MyFrac(ZZ)
P(1), P(2,3)

((1):(1), (2):(3))

In [37]:
P.zero()

(0):(1)

In [38]:
a = MyElement(P, 9,4)
b = MyElement(P, 1,2)
c = MyElement(P, -1,2)
type(P.sum([a,b,c]))

<class '__main__.MyFrac_with_category.element_class'>

In [39]:
type(sum([a,b,c]))

<class '__main__.MyElement'>

#### How did that happen?

We provided "P.Element", and thus benefit from the lazy attribute "P.element\_class".

It provides another dynamic class, a sub-class of both MyElement and of `P.category().element_class` :

In [40]:
P.__class__.element_class

In [41]:
P.element_class
type(P.element_class)

<class '__main__.MyFrac_with_category.element_class'>

<class 'sage.structure.dynamic_class.DynamicInheritComparisonMetaclass'>

In [42]:
issubclass(P.element_class, MyElement)
issubclass(P.element_class,P.category().element_class)

True

True

The **default \_\_call\_\_ method** of P passes the arguments to P.element\_class, adding the argument "parent=P".

In particular, elements obtained by calling P are instances of that new automatically created class.

In [43]:
type(P(2,3))

<class '__main__.MyFrac_with_category.element_class'>

Note: For that reason, we used `self.__class__` in the arithmetic methods of MyElement.

`P.zero_element()` merely does `P(0)` and thus returns an instance of `P.element_class`. Since `P.sum([...])` starts with `P.zero_element()`, we have:

In [44]:
type(a)
isinstance(a,P.element_class)
type(P.sum([a,b,c]))

<class '__main__.MyElement'>

False

<class '__main__.MyFrac_with_category.element_class'>

The method **factor()** inherited from P.category().element\_class (see above) simply works:

In [45]:
a
a.factor()
P(6,4).factor()

(9):(4)

2^-2 * 3^2

2^-1 * 3

That's surprising: The element "a" is just an instance of MyElement, not of P.element\_class, and its class does not know about the factor method.

In fact, we see the above-mentioned \_\_getattr\_\_ at work.

In [46]:
hasattr(type(a), 'factor')
hasattr(P.element_class, 'factor')
hasattr(a, 'factor')

False

True

True

#### Need for speed

The category framework is sometimes blamed for speed regressions (see \#9138 versus \#11900).

But if categories are **used properly, it is fast**:

In [47]:
timeit('a.factor',number=1000)
type(a)   # Here, the category framework is NOT properly used

1000 loops, best of 3: 615 ns per loop

<class '__main__.MyElement'>

In [48]:
a2 = P(9,4)          # Here, it IS properly used
a2 == a

True

In [49]:
timeit('a2.factor',number=1000); type(a2)

1000 loops, best of 3: 211 ns per loop

<class '__main__.MyFrac_with_category.element_class'>

So, *don't be afraid of using categories...*

## Coercion - the basics

> *Coercion is involved if one wants to be able to do arithmetic, comparisons, etc. between elements of distinct parents.*

### Theoretical background

#### Dissociation from *type conversion*

* In C: "coercion" means "automatic type conversion".
* In Sage: **Coercion is not about a change of types, but about a change of parents.**

Elements of the same type may have very different parents, such as here:

In [50]:
P1 = QQ['v,w']; P2 = ZZ['w,v']

In [51]:
type(P1.gen()) == type(P2.gen())
P1 == P2

True

False

P2 naturally is a sub-ring of P1. So, it makes sense to be able to add elements of the two rings - the result should then live in P1:

In [52]:
(P1.gen()+P2.gen()).parent() is P1

True

It would be rather inconvenient if one needed to *explicitly* convert an element of P2 into P1 before adding. The coercion system does that conversion automatically.

#### Coercion versus conversion

A coercion happens implicitly, without being explicitly requested by the user.

Hence, coercion must be based on mathematical rigour.

##### 1. A coercion is a map, a conversion may be a *partial* map

For example, a polynomial of degree zero over the integers can be interpreted as an integer - but there is no general conversion of a polynomial into an integer. So, that must not be a coercion.

Hence, coercion maps are defined on the level of parents, and they can be requested with the following methods:

In [53]:
P1.has_coerce_map_from(P2)

True

In [54]:
P1.coerce_map_from(P2)

Coercion map:
  From: Multivariate Polynomial Ring in w, v over Integer Ring
  To:   Multivariate Polynomial Ring in v, w over Rational Field

In [55]:
P2.has_coerce_map_from(P1)

False

##### 2. A coercion is a morphism, i.e., structure preserving

Any real number can be converted to an integer, namely by rounding. However, such a conversion is not useful in arithmetic operations:

In [56]:
int(1.6)+int(2.7) == int(1.6+2.7)

False

It depends on the parents which structure is to be preserved. For example, the coercion from the integers into the rational field is a homomorphism of euclidean domains:

In [57]:
QQ.coerce_map_from(ZZ).category()

Category of homsets of euclidean domains and metric spaces

In [58]:
QQ.coerce_map_from(ZZ).category_for()

Join of Category of euclidean domains and Category of infinite sets and Category of metric spaces

##### 3. If a coercion exists, it has to be unique

In addition, if there is a coercion from P1 to P2 and x is an element of P1, then the conversion P2(x) has to coincide with the coercion of x into P2.

##### 4. Coercions can be composed

If there is a coercion φ from P1 to P2 and another coercion ψ from P2 to P3, then the composition of φ followed by ψ must yield a (the) coercion from P1 to P3.

##### 5. The identity is a coercion

Together with the two preceding requirements, it follows: If there are conversions from P1 to P2 and from P2 to P1, then they are mutually inverse.

### Implementing conversion

We implement some conversions into our fraction fields by simply providing the attribute "Element".

However, we can not convert elements of a fraction field into elements of another fraction field, yet:

In [59]:
P(2/3)

ValueError: Numerator and denominator must be elements of Integer Ring

For implementing a conversion, **the default \_\_call\_\_ method should (almost) never be overridden.**Again, some old parents violate that rule - please help to refactor them!

Instead, **implement the method \_element\_constructor\_**, that should return an instance of the parent's element class.

In [60]:
class MyFrac(MyFrac):
    def _element_constructor_(self, *args,**kwds):
        if len(args)!=1:
            return self.element_class(self, *args, **kwds)
        x = args[0]
        try:
            P = x.parent()
        except AttributeError:
            return self.element_class(self, x, **kwds)
        if P in QuotientFields() and P != self.base():
            return self.element_class(self, x.numerator(), x.denominator(), **kwds)
        return self.element_class(self, x, **kwds)

In addition to the conversion from the base ring and from pairs of base ring elements, we now also have a conversion from the rationals to our fraction field of ZZ:

In [61]:
P = MyFrac(ZZ)
P(2), P(2,3), P(3/4)

((2):(1), (2):(3), (3):(4))

Recall that above, the test "1 in P" failed with an error. Since we can now convert the integer 1 into P, the error has vanished. But there is still a negative outcome:

In [62]:
1 in P

False

The technical reason: We have a conversion P(1) of 1 into P, but this is not a coercion.

In [63]:
P.has_coerce_map_from(ZZ)
P.has_coerce_map_from(QQ)

False

False

### Establishing a coercion

-   We *could* use P.register\_coercion during initialisation (see documentation of that method).
-   More flexible: Provide a method \_coerce\_map\_from\_

\_coerce\_map\_from\_ accepts a parent as argument. If it returns "True", then conversion is used for coercion (but it may return an actual map).

Note that we need a special case for the rational field, since QQ.base() is not the ring of integers.

In [64]:
class MyFrac(MyFrac):
    def _coerce_map_from_(self, S):
        if self.base().has_coerce_map_from(S):
            return True
        if S in QuotientFields():
            if self.base().has_coerce_map_from(S.base()):
                return True
            if hasattr(S,'ring_of_integers') and self.base().has_coerce_map_from(S.ring_of_integers()):
                return True

By the method above, a parent coercing into the base ring will also coerce into the fraction field, and a fraction field coerces into another fraction field if there is a coercion of the corresponding base rings. Now, we have:

In [65]:
P = MyFrac(QQ['x'])
P.has_coerce_map_from(ZZ['x'])
P.has_coerce_map_from(Frac(ZZ['x']))
P.has_coerce_map_from(QQ)

True

True

True

We can now use coercion from ZZ\['x'\] and from QQ into P for arithmetic operations between the two rings:

In [66]:
3/4+P(2)+ZZ['x'].gen()
(P(2)+ZZ['x'].gen()).parent() is P

(4*x + 11):(4)

True

### Equality and element containment

Recall that above, the test "1 in P" had a negative outcome. Let us repeat the test now:

In [67]:
1 in P

True

Why is that?

The default element containment test "x in P" is based on the interplay of three building blocks: conversion, coercion and equality test.

1.  Clearly, if the conversion P(x) raises an error, then x can not be seen as an element of P. On the other hand, a conversion P(x) can in general do very nasty things. So, the fact that P(x) works does not mean that x is in P.
2.  If x.parent() is P, then the conversion P(x) will not change x (at least, that's the default). Hence, we will have x==P(x).
3.  Sage uses coercion not only for arithmetic operations, but also for comparison. Hence, if there is a coercion between the parent of x and P, then it will be applied. *If* there is a coercion from x.parent() to P then x==P(x) reduces to P(x)==P(x). Otherwise, x==P(x) will evaluate as false.

That leads to the following **default implementation of element containment testing**:

<p style="margin-left: 30px">"x in P" holds if and only if "x==P(x)" does not raise an error and evaluates as true</p>

If the user is not happy with that behaviour, the "magical" Python method \_\_contains\_\_ can be overridden.

## Coercion - the advanced parts

So far, we are able to add integers and rational numbers to elements of our new implementation of the fraction field of ZZ.

In [68]:
P = MyFrac(ZZ)

In [69]:
1/2+P(2,3)+1

(13):(6)

Surprisingly, we can even add a polynomial over the integers to an element of P, even though the **result lives in a new parent**, namely in a polynomial ring over P:

In [70]:
P(1/2) + ZZ['x'].gen()
(P(1/2) + ZZ['x'].gen()).parent() is P['x']

(1):(1)*x + (1):(2)

True

In the next, seemingly more easy example, there "obviously" is a coercion from the fraction field of ZZ to the fraction field of ZZ\[x\].

However, Sage does not know enough about our new implementation of fraction fields. Hence, it does not recognise the coercion:

In [71]:
Frac(ZZ['x']).has_coerce_map_from(P)

False

#### How / why was the new ring was constructed above? How can we establish a coercion from P to Frac(ZZ\[x\]) ?

Most parents can be constructed from simpler pieces. Parents are supposed to tell how they can be constructed:

In [72]:
Poly,R = QQ['x'].construction()
Poly,R
Fract,R = QQ.construction()
Fract,R

(Poly[x], Rational Field)

(FractionField, Integer Ring)

The first return value is a *ConstructionFunctor* that yields the parent when applied to the second return value.
Indeed, we have functors

* Fract: IntegralDomains() $\to$ FractionFields()
* Poly\[x\]: Rings() $\to$ Rings()

In particular, the construction functors can be composed:

In [73]:
Poly(QQ) is QQ['x']
Poly(ZZ) is ZZ['x']
Poly(P) is P['x']

True

True

True

In [74]:
Poly*Fract

Poly[x](FractionField(...))

In [75]:
(Poly*Fract)(ZZ) is QQ['x']

True

In addition, for a *ConstructionFunctor* it is assumed that there is a coercion from input to output:

In [76]:
((Poly*Fract)(ZZ))._coerce_map_from_(ZZ)

Composite map:
  From: Integer Ring
  To:   Univariate Polynomial Ring in x over Rational Field
  Defn:   Natural morphism:
          From: Integer Ring
          To:   Rational Field
        then
          Polynomial base injection morphism:
          From: Rational Field
          To:   Univariate Polynomial Ring in x over Rational Field

Construction functors do not necessarily commute:

In [77]:
(Fract*Poly)(ZZ)
(Poly*Fract)(ZZ)

Fraction Field of Univariate Polynomial Ring in x over Integer Ring

Univariate Polynomial Ring in x over Rational Field

#### Given:

<p style="margin-left: 30px"> We have parents P1, P2, R and construction functors F1, F2, such that
P1 = F1(R) and P2 = F2(R)</p>

#### Wanted:

<p style="margin-left: 30px"> Find new construction functor F3, such that both P1 and P2 coerce into P3=F3(R).</p>

In analogy to a notion of category theory, P3 is called the *pushout* of P1 and P2; and similarly F3 is called the pushout of F1 and F2.

In [78]:
from sage.categories.pushout import pushout
pushout(Fract(ZZ),Poly(ZZ))

Univariate Polynomial Ring in x over Rational Field

F1\*F2 and F2\*F1 are natural candidates for the pushout of F1 and F2. However, the result must not depend on the order

$\Rightarrow$ We need a consistent choice of order: "indecomposable" construction functors have a **rank**.

If F1.rank is smaller thanF2.rank, then the pushout is F2\*F1 (hence, F1 is applied first).

We have

In [79]:
Fract.rank, Poly.rank

(5, 9)

and thus the pushout is

In [80]:
Fract.pushout(Poly), Poly.pushout(Fract)

(Poly[x](FractionField(...)), Poly[x](FractionField(...)))

This is why the example above has worked.

However, only "elementary" construction functors have a rank:

In [81]:
(Fract*Poly).rank

AttributeError: 'CompositeConstructionFunctor' object has no attribute 'rank'

**What do we do with two *chains* of construction functors?**

Shuffle the functors from the two chains (F1,F2,...) and (G1,G2,...):

* If F1.rank &lt; G1.rank: Apply F1, and proceed with (F2,F3,...) and (G1,G2,...).
* If F1.rank &gt; G1.rank: Apply G1, and proceed with (F1,F2,...) and (G2,G3,...).
* If F1.rank == G1.rank, then we need other techniques (see below).

As an illustration, we first get us some functors and then see how chains of functors are shuffled.

In [82]:
AlgClos, R = CC.construction(); AlgClos

AlgebraicClosureFunctor

In [83]:
Compl, R = RR.construction(); Compl

Completion[+Infinity, prec=53]

In [84]:
Matr, R = (MatrixSpace(ZZ,3)).construction(); Matr

MatrixFunctor

In [85]:
AlgClos.rank, Compl.rank, Fract.rank, Poly.rank, Matr.rank

(3, 4, 5, 9, 10)

When we apply Fract, AlgClos, Poly and Fract to the ring of integers, we obtain

In [86]:
(Fract*Poly*AlgClos*Fract)(ZZ)

Fraction Field of Univariate Polynomial Ring in x over Algebraic Field

When we apply Compl, Matr and Poly to the ring of integers, we obtain

In [87]:
(Poly*Matr*Compl)(ZZ)

Univariate Polynomial Ring in x over Full MatrixSpace of 3 by 3 dense matrices over Integer Ring

Applying the shuffling procedure yields

In [88]:
(Poly*Matr*Fract*Poly*AlgClos*Fract*Compl)(ZZ)

Univariate Polynomial Ring in x over Full MatrixSpace of 3 by 3 dense matrices over Fraction Field of Univariate Polynomial Ring in x over Algebraic Field

and this is indeed equal to the pushout found by Sage:

In [89]:
pushout((Fract*Poly*AlgClos*Fract)(ZZ), (Poly*Matr*Compl)(ZZ))

Univariate Polynomial Ring in x over Full MatrixSpace of 3 by 3 dense matrices over Fraction Field of Univariate Polynomial Ring in x over Algebraic Field

#### The case F1.rank==G1.rank

Sage's pushout constructions offers two ways to proceed:

1.  Construction functors have a method `merge()` that either returns None or returns a construction functor. If either F1.merge(G1) or G1.merge(F1) return a functor H1, then apply H1 and proceed with (F2,F3,...) and (G2,G3,...).
2.  Construction functors have a method `commutes()`. If F1.commutes(G1) or G1.commutes(F1) returns True, then apply both F1 and G1 in any order, and proceed with (F2,F3,...) and (G2,G3,...).

By default, F1.merge(G1) returns F1 if F1==G1, and returns None otherwise.

The `commutes()` method exists, but it seems that so far nobody has implemented two functors of the same rank that commute.

### A use case of  ` merge()`

#### Provide coercion between different implementations of the same algebraic structure

If F1(P) and F2(P) are different implementations of the same thing, then F1.merge(F2)(P) should be the default implementation of that thing.

Here:

-   Implement an alternative to the "usual" fraction field functor, having the same rank, but returning our new implementation.
-   Make our new implementation the default, by providing a merge method.

Remark:

-   **Do not override the default \_\_call\_\_ method** -- implement `_apply_functor` instead.
-   Declare **domain** and **codomain** of the functor.

In [90]:
from sage.categories.pushout import ConstructionFunctor
class MyFracFunctor(ConstructionFunctor):
    rank = 5
    def __init__(self):
        ConstructionFunctor.__init__(self, IntegralDomains(), Fields())
    def _apply_functor(self, R):
        return MyFrac(R)
    def merge(self, other):
        if isinstance(other, (type(self), sage.categories.pushout.FractionField)):
            return self

In [91]:
MyFracFunctor()

MyFracFunctor

We verify that our functor can really be used to construct our implementation of fraction fields, and that it can be merged with either itself or the usual fraction field constructor:

In [92]:
MyFracFunctor()(ZZ)

NewFrac(Integer Ring)

In [93]:
MyFracFunctor().merge(MyFracFunctor())

MyFracFunctor

In [94]:
MyFracFunctor().merge(Fract)

MyFracFunctor

There remains to let our new fraction fields know about the new construction functor:

In [95]:
class MyFrac(MyFrac):
    def construction(self):
        return MyFracFunctor(), self.base()

In [96]:
MyFrac(ZZ['x']).construction()

(MyFracFunctor, Univariate Polynomial Ring in x over Integer Ring)

Due to merging, we have:

In [97]:
pushout(MyFrac(ZZ['x']), Frac(QQ['x']))

NewFrac(Univariate Polynomial Ring in x over Rational Field)

Note that merging of functors really occurs here: There is no coercion from QQ\['x'\] to ZZ\['x'\].

## The Test Suites

The category framework does not only provide functionality but also a test framework.

### "Abstract" methods

A category can require/suggest certain parent or element methods, that the user must/should implement. This is in order to smoothly blend with the methods that already exist in Sage.

The methods that ought to be provided are called "abstract methods". Let us see what methods are needed for quotient fields and their elements:

In [98]:
from sage.misc.abstract_method import abstract_methods_of_class

In [99]:
abstract_methods_of_class(QuotientFields().parent_class)

{'optional': [], 'required': ['__contains__']}

Hence, the only required method (that is actually required for all parents that belong to the category of sets) is an element containment test. That's fine, because the base class $sage.structure.parent.Parent$ provides a default containment test.

The elements have to provide more:

In [100]:
abstract_methods_of_class(QuotientFields().element_class)

{'optional': ['_add_', '_mul_'],
 'required': ['__nonzero__', 'denominator', 'numerator']}

Hence, the elements must provide `denominator()` and `numerator()` methods, may provide Sage's single underscore arithmetic methods (actually any ring element *should* provide them).

#### \_test\_... methods

If a parent or element method's name start with "\_test\_", it gives rise to a test in the automatic test suite.

For example, it is tested

-   whether a parent P actually is an instance of the parent class of the category of P,
-   whether the user hase implemented the required abstract methods,
-   whether some defining structural properties (e.g., commutativity) hold.

Here are the tests that form the test suite of quotient fields:

In [101]:
[t for t in dir(QuotientFields().parent_class) if t.startswith('_test_')]

['_test_additive_associativity',
 '_test_an_element',
 '_test_associativity',
 '_test_cardinality',
 '_test_characteristic',
 '_test_characteristic_fields',
 '_test_distributivity',
 '_test_divides',
 '_test_elements',
 '_test_elements_eq_reflexive',
 '_test_elements_eq_symmetric',
 '_test_elements_eq_transitive',
 '_test_elements_neq',
 '_test_euclidean_degree',
 '_test_fraction_field',
 '_test_gcd_vs_xgcd',
 '_test_one',
 '_test_prod',
 '_test_quo_rem',
 '_test_some_elements',
 '_test_zero',
 '_test_zero_divisors']

Since we have implemented all abstract methods, we are confident that the test suite runs without complaining. In fact, it does!

In [102]:
P = MyFrac(ZZ['x'])

In [103]:
TestSuite(P).run()

Let us see what tests are actually performed:

In [104]:
TestSuite(P).run(verbose=True)

running ._test_additive_associativity() . . . pass
running ._test_an_element() . . . pass
running ._test_associativity() . . . pass
running ._test_cardinality() . . . pass
running ._test_category() . . . pass
running ._test_characteristic() . . . pass
running ._test_characteristic_fields() . . . pass
running ._test_distributivity() . . . pass
running ._test_divides() . . . pass
running ._test_elements() . . .
  Running the test suite of self.an_element()
  running ._test_category() . . . pass
  running ._test_eq() . . . pass
  running ._test_new() . . . pass
  running ._test_nonzero_equal() . . . pass
  running ._test_not_implemented_methods() . . . pass
  running ._test_pickling() . . . pass
  pass
running ._test_elements_eq_reflexive() . . . pass
running ._test_elements_eq_symmetric() . . . pass
running ._test_elements_eq_transitive() . . . pass
running ._test_elements_neq() . . . pass
running ._test_eq() . . . pass
running ._test_euclidean_degree() . . . pass
running ._test_fraction

As one can see, tests are also performed on elements. There are methods that return one element or a list of some elements, relying on "typical" elements that can be found in most algebraic structures.

In [105]:
P.an_element(); P.some_elements()

(2):(1)

[(2):(1)]

Unfortunately, the list of elements that is returned by the default method is of length one, and that single element could also be a bit more interesting.

The method an\_element relies on a method \_an\_element\_, so, we implement that. We also override the some\_elements method.

In [106]:
class MyFrac(MyFrac):
    def _an_element_(self):
        a = self.base().an_element()
        b = self.base_ring().an_element()
        if (a+b)!=0:
            return self(a)**2/(self(a+b)**3)
        if b != 0:
            return self(a)/self(b)**2
        return self(a)**2*self(b)**3
    def some_elements(self):
        return [self.an_element(),self(self.base().an_element()),self(self.base_ring().an_element())]

In [107]:
P = MyFrac(ZZ['x'])
P.an_element(); P.some_elements()

(x^2):(x^3 + 3*x^2 + 3*x + 1)

[(x^2):(x^3 + 3*x^2 + 3*x + 1), (x):(1), (1):(1)]

Now, as we have more interesting elements, we may also add a test for the "factor" method. Recall that the method was inherited from the category, but it appears that it is not tested.

Normally, a test for a method defined by a category should be provided by the same category. Hence, since `factor()` is defined in the category of quotient fields, a test should be added there. But we won't change source code here and will instead create a sub-category.

Apparently, the product of the factors returned be `e.factor()` should be equal to `e`. For forming the product, we use the `prod()` method, that, no surprise, is inherited from another category.

In [108]:
P.prod.__module__

'sage.categories.monoids'

When we want to create a sub-category, we need to provide a method `super_categories()`, that returns a list of all immediate super categories (here: category of quotient fields). The parent and element methods of a category are provided as methods of a class `ParentMethods` and `ElementMethods`, as follows:

In [109]:
from sage.categories.category import Category
class QuotientFieldsWithTest(Category): # do *not* inherit from QuotientFields, but ...
    def super_categories(self):
        return [QuotientFields()]       # ... declare QuotientFields as a super category!
    class ParentMethods:
        pass
    class ElementMethods:
        def _test_factorisation(self, **options):
            P = self.parent()
            assert self == P.prod([P(b)**e for b,e in self.factor()])

We provide an instance of our quotient field implementation with that new category. Note that categories have a default \_repr\_ method, that guesses a good string representation from the name of the class: QuotientFieldsWithTest becomes "quotient fields with test".

In [110]:
P = MyFrac(ZZ['x'], category=QuotientFieldsWithTest())
P.category()

Category of quotient fields with test

The new test is inherited from the category. Since an\_element() is returning a complicated element, \_test\_factorisation is a serious test.

In [111]:
P.an_element()._test_factorisation

<bound method MyFrac_with_category.element_class._test_factorisation of (x^2):(x^3 + 3*x^2 + 3*x + 1)>

In [112]:
P.an_element().factor()

(x + 1)^-3 * x^2

Last, we observe that the new test has automatically become part of the test suite. We remark that the existing test became more serious as well, by an\_element() returning something more interesting.

In [113]:
TestSuite(P).run(verbose=True)

running ._test_additive_associativity() . . . pass
running ._test_an_element() . . . pass
running ._test_associativity() . . . pass
running ._test_cardinality() . . . pass
running ._test_category() . . . pass
running ._test_characteristic() . . . pass
running ._test_characteristic_fields() . . . pass
running ._test_distributivity() . . . pass
running ._test_divides() . . . pass
running ._test_elements() . . .
  Running the test suite of self.an_element()
  running ._test_category() . . . pass
  running ._test_eq() . . . pass
  running ._test_factorisation() . . . pass
  running ._test_new() . . . pass
  running ._test_nonzero_equal() . . . pass
  running ._test_not_implemented_methods() . . . pass
  running ._test_pickling() . . . pass
  pass
running ._test_elements_eq_reflexive() . . . pass
running ._test_elements_eq_symmetric() . . . pass
running ._test_elements_eq_transitive() . . . pass
running ._test_elements_neq() . . . pass
running ._test_eq() . . . pass
running ._test_euclidean